# Traverse LCP, then MFS

* Replace earlier masked-array strategy with regular arrays, using 0 to represent a null.
* Real data (offset of token within witness) is one-based.



In [926]:
from typing import List
from linsuffarr import SuffixArray
from linsuffarr import UNIT_BYTE
import pprint
import numpy as np
import re
from dataclasses import dataclass
from heapq import * # priority heap, https://docs.python.org/3/library/heapq.html
pp = pprint.PrettyPrinter(indent=2)
from bisect import bisect_right
from IPython.display import display, HTML, SVG
import graphviz
from collections import deque
debug = False

In [927]:
sigla = ['w0', 'w1', 'w2', 'w3', 'w4', 'w5']
filenames = ['darwin/darwin1859.txt', 'darwin/darwin1860.txt', 'darwin/darwin1861.txt', 'darwin/darwin1866.txt', 'darwin/darwin1869.txt', 'darwin/darwin1872.txt']
# sigla = ['w0', 'w1', 'w2', 'w3']
# filenames = ['darwin/darwin1859.txt', 'darwin/darwin1860.txt', 'darwin/darwin1861.txt', 'darwin/darwin1866.txt']
# sigla = ['w0', 'w1']
# filenames = ['darwin1859.txt', 'darwin1860.txt']
# sigla = ['w0', 'w1', 'w2', 'w3', 'w4', 'w5']
# filenames = ['abc/abcd.txt', 'abc/abcda.txt', 'abc/abcdb.txt', 'abc/abcdc.txt', 'abc/abcdd.txt', 'abc/abcde.txt']
first_paragraph = 0
last_paragraph = 50
how_many_paragraphs = last_paragraph - first_paragraph
raw_data_dict = {}
for siglum, filename in zip(sigla, filenames):
    with open(filename) as f:
        lines = f.readlines()
        lines = [line for line in lines if line != '\n']
        raw_data_dict[siglum] = " ".join(lines[first_paragraph : last_paragraph])

In [928]:
def tokenize_witnesses(witness_strings: List[str]): # one string per witness
    '''Return list of witnesses, each represented by a list of tokens'''
    # TODO: handle punctuation, upper- vs lowercase
    witnesses = []
    for witness_string in witness_strings:
        # witness_tokens = witness_string.split()
        witness_tokens = re.findall(r'\w+\s*|\W+', witness_string)
        witness_tokens = [token.strip().lower() for token in witness_tokens]
        witnesses.append(witness_tokens)
    return witnesses

In [929]:
def create_token_array(_witness_token_lists): # list of token lists per witness
    '''Create token array (single list, with separator " # " between witnesses'''
    _token_array = [] # strings
    _token_membership_array = [] # witness identifiers, same offsets as in token_array
    _token_witness_offset_array = [] # one-based offset of token in witness
    _last_witness_offset = len(_witness_token_lists) - 1
    for _index, _witness_token_list in enumerate(_witness_token_lists):
        _token_array.extend(_witness_token_list)
        for _token_offset, _token in enumerate(_witness_token_list): # don't need enumerate, just len()
            _token_witness_offset_array.append(_token_offset)
        _token_membership_array.extend([_index for _token in _witness_token_list])
        if _index < _last_witness_offset:
            _separator = " #" + str(_index + 1) + " "
            _token_array.append(_separator)
            _token_membership_array.append(_separator)
            _token_witness_offset_array.append(-1)
    return _token_array, _token_membership_array, _token_witness_offset_array

In [930]:
witness_sigla = [key for key in raw_data_dict.keys()]
witnesses = tokenize_witnesses([value for value in raw_data_dict.values()]) # strings
# token_list

In [931]:
token_array, token_membership_array, token_witness_offset_array = create_token_array(witnesses)
if debug:
    print(f"{token_array=}")
    print(f"{token_membership_array=}")
    print(f"{token_witness_offset_array=}")

In [932]:
suffix_array = SuffixArray(token_array, unit=UNIT_BYTE)
# print(suffix_array)
# LCP=0 means that the block has nothing in common with the next one

In [933]:
lcp_array = suffix_array._LCP_values
if debug:
    print(lcp_array[:5])

In [934]:
# create Block dataclass
@dataclass(unsafe_hash=True)
class Block:
    token_count: int
    start_position: int # offset into suffix array (not into token array!)
    end_position: int # start and end position give number of occurrences
    all_start_positions: [] # compute after blocks have been completed
    witnesses: set
    witness_count: int # number of witnesses in which pattern occurs, omitted temporarily because requires further computation
    frequency: int # number of times pattern occurs in whole witness set (may be more than once in a witness), end_position - start_position + 1
    # how_created: int # debug

In [935]:
@dataclass
class Lcp_interval_candidate:
    lcp_start_offset: int
    lcp_interval_token_count: int
    lcp_end_offset: int = -1

In [936]:
def check_for_depth_and_repetition(_suffix_array, _token_membership_array, _lcp_interval:Lcp_interval_candidate, _witness_count: int) -> bool:
    """Write a docstring someday

    Number of prefixes >= total number of witnesses
    Accumulate set of witness sigla for prefixes
    if:
        no witness occurs more than once, return True to keep this block
    else:
        return False
    """
    block_instance_count = _lcp_interval.lcp_end_offset - _lcp_interval.lcp_start_offset + 1
    if block_instance_count != _witness_count:
        return False
    else:
        _witnesses_found = []
        for _lcp_interval_item_offset in range(_lcp_interval.lcp_start_offset, _lcp_interval.lcp_end_offset + 1):
            _token_position = _suffix_array.SA[_lcp_interval_item_offset] # point from prefix to suffix array position
            _witness_siglum = _token_membership_array[_token_position] # point from token array position to witness identifier
            if _witness_siglum in _witnesses_found:
                return False
            else:
                _witnesses_found.append(_witness_siglum)
        return True

In [937]:
def create_blocks(_suffix_array, _token_membership_array, _witnesses, _lcp_array: list):
    """Write a docstring someday

    Look at changes in length of LCP array
    Initial value is 0 or -1 because it's a comparison with previous, and first has no previous
    Next value is number of tokens shared with previous
    Exact length doesn't matter, but if it changes, new pattern:
        If it stays the same, take note but do nothing yet; it means that the pattern repeats
        No change for a while, then goes to 0:
            Number of repetitions plus 1, e.g., 5 5 5 0 = 4 instances of 5
            Once it changes to 0, we've seen complete pattern
        Changer to smaller means hidden, deeper block
        Changes to longer means ???
    """
    _accumulator = [] # lcp positions (not values) since most recent 0
    _frequent_sequences = [] # lcp intervals to be considered for mfs
    #
    # lcp value
    # if == 0 it's a new interval, so:
    #   1. if there is already an accumulation, commit (process) it
    #      "committing the buffer" means checking for repetition and depth
    #          if it passes check: store in mfs list
    #          otherwise throw it away
    #   2. clear buffer (accumulator) and begin accumulating new buffer with the new offset with 0 value
    # otherwise it isn't zero, so there must be a buffer in place, so add to it (for now)
    for _offset, _value in enumerate(_lcp_array):
        if not _accumulator and _value == 0: # if accumulator is empty and new value is 0, do nothing
            continue
        elif not _accumulator: # accumulator is empty and new value is non-zero, so begin new accumulator
            _accumulator.append(Lcp_interval_candidate(lcp_start_offset = _offset - 1, lcp_interval_token_count = _value))
        elif _value > _accumulator[-1].lcp_interval_token_count: # new interval, so add to accumulator and continue
            _accumulator.append(Lcp_interval_candidate(lcp_start_offset = _offset - 1, lcp_interval_token_count = _value))
        elif _value == _accumulator[-1].lcp_interval_token_count: # same block as before, so do nothing
            continue
        else: # new value is less than top of accumulator, so pop everything that is higher
            # Positions in lcp array and suffix array coincide:
            #   The lcp array value is the length of the sequence
            #   The suffix array value is the start position of the sequence
            # Assume accumulator values (offsets into lcp array) point to [3, 6] and new value is 4, so:
            #   First: Pop pointer to 6 (length value in lcp array), store in frequent_sequences
            #   Second: Push new pointer to same position in lcp array, but change value in lcp array to 4
            while _accumulator and _accumulator[-1].lcp_interval_token_count > _value:
                # Create pointer to last closed block that is not filtered (like frequent_sequences)
                _newly_closed_block = _accumulator.pop()
                _newly_closed_block.lcp_end_offset = _offset - 1
                if check_for_depth_and_repetition(_suffix_array, _token_membership_array, _newly_closed_block, len(_witnesses)):
                    _frequent_sequences.append([_newly_closed_block.lcp_start_offset, _newly_closed_block.lcp_end_offset, _newly_closed_block.lcp_interval_token_count])
            # There are three options:
            #   1. there is content in the accumulator and latest value is not 0
            #   2. accumulator is empty and latest value is 0
            #   3. accumulator is empty and latest value is not 0
            # (the fourth logical combination, content in the accumulator and 0 value, cannot occur
            #     because a 0 value will empty the accumulator)
            if _value > 0 and (not _accumulator or _accumulator[-1].lcp_interval_token_count != _value):
                _accumulator.append(Lcp_interval_candidate(lcp_start_offset = _newly_closed_block.lcp_start_offset, lcp_interval_token_count = _value))
    # End of lcp array; run through any residual accumulator values
    while _accumulator:
        _newly_closed_block = _accumulator.pop()
        _newly_closed_block.lcp_end_offset = len(_lcp_array) - 1
        if check_for_depth_and_repetition(_suffix_array, _token_membership_array, _newly_closed_block, len(witnesses)):
            _frequent_sequences.append([_newly_closed_block.lcp_start_offset, len(_lcp_array)-1, _newly_closed_block.lcp_interval_token_count])
    return _frequent_sequences

In [938]:
# frequent_sequences is a list of lists
# the embedded lists contain LCP indices
#   LCP indices point into LCP array, but same index also points into suffix array
#   value in LCP array points to prefix length (compared to previous one)
#   value in suffix array points into token array
frequent_sequences = create_blocks(suffix_array, token_membership_array, witnesses, lcp_array)

In [939]:
# To remove embedded prefixes:
#
# 1. Create dictionary with end position in witness 0 (arbitrarily) as key
# 2. Set value of key to longest sequence with that end position
# 3. Dictionary values will contain only longest frequent sequences, removing embedded ones,
#    as tuples if (length, [token start positions for all witnesses])

@dataclass
class LongestSequence:
    length: int
    witness_start_and_end: List[int]

def find_longest_sequences(_frequent_sequences, _suffix_array):
    _largest_blocks = {} # key is token end position, value is (length, [witness-start-positions])
    for _frequent_sequence in _frequent_sequences:
        _length = _frequent_sequence[2]
        _suffix_array_values = [_suffix_array.SA[i] for i in range(_frequent_sequence[0], _frequent_sequence[1] + 1)]
        _token_end_position = min(_suffix_array_values) + _length # token end position for first witness
        if _token_end_position not in _largest_blocks: # first block with this end position, so create new key
            _largest_blocks[_token_end_position] = (_length, sorted(_suffix_array_values))
        else: # if new block is longer, replace old one with same key
            if _length > _largest_blocks[_token_end_position][0]:
                _largest_blocks[_token_end_position] = (_length, sorted(_suffix_array_values))
    return _largest_blocks

largest_blocks = find_longest_sequences(frequent_sequences, suffix_array)
if debug:
    print(f"{largest_blocks=}")

In [940]:
def prepare_for_beam_search(_witnesses, _token_membership_array, _largest_blocks):
    # block_offsets_by_witness: list of lists holds sorted start offsets per witness (offsets are into global token array)
    # witness_offsets_to_blocks: dictionary points from start offsets to blocks
    # score_by_block: number of tokens placed or skipped if block is placed
    # Beam search requires us, given an offset in a witness, to find the next block. We do
    #   that by looking up the value in block_offsets_by_witness and then using that value
    #   to retrieve the block key from witness_offsets_to_blocks
    # Lookup in the list of lists is:
    #   block_offsets_by_witness[witness_number][bisect_right(block_offsets_by_witness[witness_number], most_recent_offset_in_witness)]
    # (See: https://www.geeksforgeeks.org/python-find-smallest-element-greater-than-k/)
    # FIXME: traverse largest_blocks only once and add values for all witnesses in same pass
    _witness_count = len(_witnesses)
    _block_offsets_by_witness = []
    _witness_offsets_to_blocks = {}
    _first_token_offset_in_block_by_witness = [] # only tokens in blocks
    _first_absolute_token_by_witness = [] # all tokens, whether in block or not
    for i in range(_witness_count):
        _first_token_offset_in_block_by_witness.append(_token_membership_array.index(i))
        # Score = number of tokens either placed or skipped (we don't care which)
        # Low score is best because it leaves the highest potential
        # NB: The name "score" seems to imply that higher is better, and the
        #   opposite is the case here. Rename the variable?
        # NB: High potential is paramount during beam search, but should the
        #   difference between placed and skip matter at a later stage? Or
        #   does placing more blocks (more tiers) take care of that?
        _score_by_block = {}
        for i in range(_witness_count):
            _witness_offset_list = []
            for _key, _value in _largest_blocks.items():
                _witness_offset_list.append(_value[1][i])
                _witness_offsets_to_blocks[_value[1][i]] = _key
            _witness_offset_list.sort()
            _block_offsets_by_witness.append(_witness_offset_list)
    for i in range(_witness_count):
        _first_absolute_token_by_witness.append(_token_membership_array.index(i))
    for _key, _value in _largest_blocks.items():
        # to determine number of tokens that will have been placed or skipped
        #   after placing block:
        #       matrix-subtract first_token_offset_by_witness from value[1]
        #       add witness_count * value[0] (to account for block length)
        #   key by block key, value is score
        _differences = [x - y for x, y in zip(_value[1], _first_token_offset_in_block_by_witness)]
        if debug:
            print(_differences)
        _score = sum(_differences) + _witness_count * _value[0]
        _score_by_block[_key] = _score
    if debug:
        print(f"{_block_offsets_by_witness=}")
        print()
        _witness_offsets_to_blocks = { key: _witness_offsets_to_blocks[_key] for _key in sorted(_witness_offsets_to_blocks.keys())}
        print(f"{_witness_offsets_to_blocks=}")
        print()
        print(f"{_first_token_offset_in_block_by_witness=}")
        print()
        print(f"{_first_absolute_token_by_witness=}")
        print()
        print(f"{_score_by_block=}")
    return _block_offsets_by_witness, _witness_offsets_to_blocks, _first_token_offset_in_block_by_witness, _first_absolute_token_by_witness, _score_by_block

block_offsets_by_witness, witness_offsets_to_blocks, first_token_offset_in_block_by_witness, first_absolute_token_by_witness, score_by_block \
    = prepare_for_beam_search(witnesses, token_membership_array, largest_blocks)

In [941]:
# To perform beam search
#   Create single start option (at Start node, which is a fiction [there is no Start block]
#       created for the beam search)
#   Loop: for each BeamOption on current tier
#       Evaluate score for advancing in each witness and bringing others into alignment with it
#       For β lowest (!) scores create new BeamOption (this advances to next tier)
#           Score is count of tokens placed or skipped (!)
#           Favor lowest score because that has the greatest potential

In [942]:
@dataclass(order=True, frozen=True, eq=True) # heapqueue is priority queue, so requires comparison
class BeamOption:
    score: int
    path: tuple # path through sequence of blocks leading to current BeamOption

In [943]:
# Create initial BeamOption
initial = [BeamOption(score=0, path=())] # tier 0, one-item list
def perform_beam_search_step(_witnesses, _largest_blocks, _block_offsets_by_witness, _witness_offsets_to_blocks, _score_by_block, _beam_options=initial, _beta=3):
    # TODO: The witness count should be a global constant
    # print("New tier with " + str(len(beam_options)) + " beam options")
    _new_options = [] # candidates for next tier
    _finished_options = []
    for _beam_option in _beam_options:
        # ###
        # 2022-09-06
        # Three possibilities for an individual beam option:
        # 1. Option leads to new option
        # 2. Option is finished
        # 3. No new option but option isn't finished (transposition)
        # NB: We check each witness in the beam option, and if any witness
        #     raises an IndexError, the whole block cannot be advanced and
        #     is finished. (This is true because of our constraints: every
        #     block is a) full-depth and b) no repetition.)
        #
        # What to do:
        #
        # 1. Perform the bisect for each witness based on the head of the
        #    path of the current beam option. This returns an offset into
        #    the witness-specific list of block offsets. Initialize a
        #    counter to 0.
        # 2. Using the initial offsets returned by the bisect operation that
        #    we performed in step #1 (and never perform again) for each witness
        #    plus the counter (which we will increment if needed in the inner
        #    loop), check that next option for each witness. There are three
        #    possibilities for each counter value (over the entire witness group):
        #    a) If the next block (returned by this method) would overrun
        #       for any witness, it will overrun for all witnesses, so the
        #       beam option can be added to the finished list and we exit the
        #       outer loop (the one that processes the beam option).
        #    b) If the next block is a viable option, add it to the options and
        #       check the next witness within this same inner loop instance
        #       because in case of transposition different blocks will suggest
        #       different next blocks, all of which could be viable options.
        #       This ends the processing for that beam option.
        #    c) If we don't find any viable next option and don't overrun for
        #       any witness, increment the counter and replay step #2 (inner
        #       loop).
        # Exit condition: Eventually we either find a viable option or overrun.
        #
        # TODO: How should we implement this to terminate the correct loop in
        # the right place? For? While? Generator? For and while start with the
        # outer loop and work inward; with a generator we start with the inner
        # and work outward.
        # ###
        _witness_count = len(_witnesses)
        _new_finished_option_check = False
        _new_viable_option_check = 0
        _counter = 0
        while True:
            for i in range(_witness_count): # advance for each witness in turn
                if not _beam_option.path: # path is empty only for initial state at tier 0
                    _last_offset = -1 # NB: same for all witnesses, and not 0, which will break for witness 0
                else:
                    _last_offset = _largest_blocks[_beam_option.path[0]][1][i]
                try:
                    _next_offset = bisect_right(_block_offsets_by_witness[i], _last_offset)
                    _next_value = _block_offsets_by_witness[i][_next_offset + _counter]
                    _next_block = _witness_offsets_to_blocks[_next_value] # find that next block to get its length
                    # would any witness pointer move backwards?
                    # perform matrix subtraction; if signs differ, there are items that move in opposite directions
                    # first option cannot be transposed, so accept it automatically
                    if (not _beam_option.path) or (len(set([np.sign(x - y) for x, y in zip(_largest_blocks[_next_block][1], _largest_blocks[_beam_option.path[0]][1])])) == 1):
                        _new_score = _score_by_block[_next_block] # accounts for all witnesses
                        # concatenate tuples with a +;  most recent first (for priority heap)
                        _new_options.append(BeamOption(score = _new_score, path=((_next_block,) + _beam_option.path)))
                        _new_viable_option_check += 1
                    else:
                        continue
                        # print('Transposition detected for beam option:', beam_option)
                except IndexError: # we've gone as far as we can with this path
                    _new_finished_option_check = True
                    _finished_options.append(_beam_option)
                    break # if one witness overruns, they all will, so this beam option is done
            if _new_viable_option_check >= _witness_count or _new_finished_option_check:
                break
            _counter += 1
    _new_options = list(set(_new_options)) # deduplicate
    heapify(_new_options) # sort from low score to high (low score is best)
    # print(_beam_options)
    if not _new_options and not _finished_options:
        raise Exception("This shouldn't happen: no new options and no finished options")
    else:
        return _new_options[:_beta], _finished_options

In [944]:
options, _ = perform_beam_search_step(witnesses, largest_blocks, block_offsets_by_witness, witness_offsets_to_blocks, score_by_block)
finished = [] # options that cannot go further
counter = 0
while options: # no more options means that we're done
    # TODO: The beam size at the moment is a magic number; can we rationalize it?
    options, end_of_life = perform_beam_search_step(witnesses, largest_blocks, block_offsets_by_witness, witness_offsets_to_blocks, score_by_block, _beam_options=options, _beta=20)
    finished.extend(end_of_life) # add any options that cannot go further
    print(counter, len(options), len(finished))
    counter += 1
finished = list(set(finished)) # TODO: Remove this because we'll sort later?
# TODO: Verify that better scores are better alignments (how??)

0 1 0
1 1 0
2 1 0
3 1 0
4 1 0
5 1 0
6 1 0
7 1 0
8 2 0
9 4 0
10 6 0
11 11 0
12 18 0
13 20 0
14 20 0
15 20 0
16 20 0
17 20 0
18 20 0
19 20 0
20 20 0
21 20 0
22 20 0
23 20 0
24 20 0
25 20 0
26 20 0
27 20 0
28 20 0
29 20 0
30 20 0
31 20 0
32 20 2
33 19 4
34 19 5
35 20 5
36 20 5
37 20 5
38 20 5
39 20 5
40 20 5
41 20 5
42 20 5
43 20 5
44 20 5
45 20 5
46 20 5
47 20 5
48 20 5
49 20 5
50 20 5
51 20 5
52 20 5
53 20 5
54 20 5
55 20 5
56 20 5
57 20 5
58 20 5
59 20 5
60 20 5
61 20 5
62 20 5
63 20 5
64 20 5
65 20 5
66 20 5
67 20 5
68 20 5
69 20 5
70 20 5
71 20 5
72 20 5
73 20 5
74 20 5
75 20 5
76 20 5
77 20 5
78 20 5
79 20 5
80 20 5
81 20 5
82 20 5
83 20 5
84 20 5
85 20 5
86 20 5
87 20 5
88 20 5
89 20 5
90 20 5
91 20 5
92 20 5
93 20 5
94 20 5
95 20 5
96 20 5
97 20 5
98 20 5
99 20 5
100 20 5
101 20 5
102 20 5
103 20 5
104 20 5
105 20 5
106 20 5
107 20 5
108 20 5
109 20 5
110 20 5
111 20 5
112 20 5
113 20 5
114 20 5
115 20 5
116 20 5
117 20 5
118 20 5
119 20 5
120 20 5
121 20 5
122 20 5
123 20 5
124 2

In [945]:
# finished holds beam options that cannot go further, with duplicates removed
# BeamOption.score counts tokens placed or skipped, which is correct for traversing, but
#   for evaluation we count only most tokens placed and sub-sort by fewest blocks
# Blocks know their length, so we sum the lengths of the finalists and keep only the highest
# NB: There could be more than one
finished.sort(reverse = True, key = lambda f: (sum([largest_blocks[b][0] for b in f.path]), -1 * len(f.path)))
if debug:
    for pos, f in enumerate(finished):
        print(pos, sum([largest_blocks[b][0] for b in f.path]), len(f.path))
        print(f)
# TODO: Adaptive beam width? We can't evaluate the consequences of a suboptimal
# first-pass alignment until we follow through to a full alignment. It could be
# that saving time by sacrificing a small improvement in the first pass won't
# affect the outcome because we'll inevitably and at no addition cost fix it
# later.

In [946]:
# FIXME 2022-09-20:
# We find unaligned tokens by looking between blocks, as well as leading unaligned tokens.
# TODO:
#  1. We don't find unaligned tokens after the last block.
#  2. We don't test what happens if there are no unaligned tokens before the first block.
table_top = """
    <html>
        <head>
            <style type="text/css">
                table, tr, th, td {border: 1px solid black; border-collapse: collapse;}
                th, td {padding: 3px;}
                td:first-child {text-align: right;}
            </style></head><body><table><tr style="background-color: pink;"><th>Row</th>
    """ + '\n'.join(['<th style="border: 1px black solid; border-collapse: collapse; text-align: center;">w' + str(i) + '</th>' for i in range(len(witnesses))]) + '</tr>'
table_bottom = '</table></body></html>'

block0_start_positions = largest_blocks[finished[0].path[-1]][1]
# if debug:
#     print(block0_start_positions)
#     print(first_absolute_token_by_witness)
if block0_start_positions != first_absolute_token_by_witness:
    leading_unaligned_tokens = ['<td>' + " ".join(token_array[i: j]) + '</td>' for i, j in zip(first_absolute_token_by_witness, block0_start_positions)]
    leading_unaligned_row = '<tr style="background-color: lightgray; border: 1px black solid; border-collapse: collapse;"><td style="background-color: pink;">unaligned</td>' + "".join(leading_unaligned_tokens) + '</tr>'

rows = []
# Rows with aligned tokens are the same in all witness by definition
# The path contains largest_blocks keys, which represent the last token of
#   a block in witness 0
# The value of a block is a tuple, the first member of which is the length
# We can retrieve the aligned tokens by slicing them from the token_array
for index, end_token_offset in enumerate(finished[0].path[::-1]): # path is ordered from last to first
    # ###
    # Information for aligned block
    # This is the same for all witnesses, taken from witness 0
    # ###
    block_length = largest_blocks[end_token_offset][0]
    start_token_offset = end_token_offset - block_length
    tokens = token_array[start_token_offset: end_token_offset]
    # ###
    # Information for preceding non-aligned block
    # This is different for each witness
    #
    # Loop over witnesses using range(len(witnesses))
    # Get start token offset for aligned block for current witness
    # Get end token offset for preceding aligned block for current witness
    # Get tokens by slicing token array
    # ###
    if index > 0:
        current_block = largest_blocks[end_token_offset]
        preceding_block = largest_blocks[finished[0].path[::-1][index - 1]]
        unaligned_row = []
        unaligned_row.append('<tr style="background-color: lightgray; border: 1px black solid; border-collapse: collapse;"><td style="background-color: pink;">unaligned</td>')
        for i in range(len(witnesses)):
            unaligned_start_token_offset = preceding_block[1][i] + preceding_block[0]
            unaligned_end_token_offset = current_block[1][i] - 1
            unaligned_tokens = token_array[unaligned_start_token_offset: unaligned_end_token_offset + 1]
            unaligned_row.append('<td style="border: 1px black solid; border-collapse: collapse;">' + " ".join(unaligned_tokens) + '</td>')
        unaligned_row.append('</tr>')
        rows.append("".join(unaligned_row))
    # ###
    # Create aligned block
    # ###
    rows.append('<tr style="background-color: beige; border: 1px black solid; border-collapse: collapse;"><td style="background-color: pink; border: 1px black solid; border-collapse: collapse;">' + str(index) + ' (' + str(end_token_offset) + ')</td><td  style="border: 1px black solid; border-collapse: collapse;" colspan="' + str(len(witnesses)) + '">' + " ".join(tokens) + '</td></tr>')
table = table_top + leading_unaligned_row + "".join(rows) + table_bottom
with open('table-output.html', 'w') as f:
    f.write(table)
HTML(table)

In [947]:
# # Create hybrid graph
# # Aligned blocks are a single node with (except if they are initial or last)
# #   separate in- and out-edges for each witness.
# # Unaligned blocks are separate nodes for each witness that contained all
# #   unaligned cells at that location.
# # The aligned blocks are treated as if they were in a variant graph.
# # Unaligned blocks can then be processed to look for new alignments (made
# #   possible because the first pass removed repetition. Initially we can
# #   complete the graph after the initial pass and then traverse it to
# #   process the unaligned portions. Eventually we should process them in
# #   place, recursively, as we construct the graph.
# # Aligned blocks have node numbers a0, a1, a2, etc.
# # Unaligned token sequences have node numbers u0, u1, u2, etc.
# # START node is numbered -1
# # END node is numbered
#
# # Initialize graph
# a = graphviz.Digraph(format="svg", graph_attr={'rankdir': 'LR'})
# unaligned_node_no = -1 # Node id values are integers except for START and END
# aligned_node_no = 0
# preceding_aligned_node_no = -1
# a.node("".join(('a', str(aligned_node_no))), "".join(('a', str(aligned_node_no))) + ': ' + 'START')
#
# # Blocks are ordered backwards, so start from end
# block0_start_positions = largest_blocks[finished[0].path[-1]][1]
#
# # Add all aligned blocks as nodes
# for index, end_token_offset in enumerate(finished[0].path[::-1]):
#     # For aligned block increment node, aligned_node, and preceding_aligned_node
#     aligned_node_no += 1
#     aligned_id = "".join(('a', str(aligned_node_no)))
#     preceding_aligned_node_no += 1
#     preceding_aligned_id = "".join(('a', str(preceding_aligned_node_no)))
#     # Same tokens for all witnesses, so get them from witness 0
#     block_length = largest_blocks[end_token_offset][0]
#     start_token_offset = end_token_offset - block_length
#     tokens = token_array[start_token_offset: end_token_offset]
#     a.node(aligned_id, aligned_id + ": " + " ".join(tokens))
#
#     # Check for initial unaligned tokens
#     # TODO: We assume leading tokens, and probably don't handle their absence correctly
#     # TODO: Move this out of the loop; it can happen only once
#     if aligned_node_no == 1: # only for first aligned block
#         for i, j in zip(first_absolute_token_by_witness, block0_start_positions):
#             unaligned_node_no += 1
#             unaligned_id = "".join(('u', str(unaligned_node_no)))
#             unaligned_tokens = " ".join(token_array[i: j+1])
#             a.node(unaligned_id, unaligned_id + ': ' + unaligned_tokens)
#             a.edge(preceding_aligned_id, unaligned_id)
#             a.edge(unaligned_id, aligned_id)
#
#     # Add unaligned tokens between last aligned block (preceding_node) and new one (node_id)
#     if index > 0:
#         current_block = largest_blocks[end_token_offset]
#         preceding_block = largest_blocks[finished[0].path[::-1][index - 1]]
#         for i in range(len(witnesses)):
#             unaligned_node_no += 1
#             unaligned_start_token_offset = preceding_block[1][i] + preceding_block[0]
#             unaligned_end_token_offset = current_block[1][i] - 1
#             unaligned_tokens = token_array[unaligned_start_token_offset: unaligned_end_token_offset + 1]
#             unaligned_id = "".join(('u', str(unaligned_node_no)))
#             a.node(unaligned_id, unaligned_id + ": " + " ".join(unaligned_tokens))
#             a.edge(preceding_aligned_id, unaligned_id)
#             a.edge(unaligned_id, aligned_id)
#
# # Add END node
# # TODO: We assume no trailing tokens and don't look for or handle them
# # TODO: Edge connections for END node will have to be revised once we
# #   also deal with trailing unaligned tokens
# aligned_node_no += 1
# aligned_id = "".join(('a', str(aligned_node_no)))
# preceding_aligned_node_no += 1
# preceding_aligned_id = "".join(('a', str(preceding_aligned_node_no)))
# a.node(aligned_id, aligned_id + ": " + 'END')
# a.edge(preceding_aligned_id, aligned_id)
#
# # Render graph
# svg = a.render()
# # display(SVG(svg))
#
# # Traverse aligned and unaligned intervals as we do with alignment table, above,
# #   creating nodes and edges as we proceed.
# # TODO: Currently we assume unaligned tokens at the beginning and no unaligned
# #   tokens at the end. This is correct for our sample but not reliable for
# #   arbitrary input.

In [948]:
# Create tree to represent alignment
@dataclass
class Node:
    """Nodes of all types have integer id properties"""
    id: int

@dataclass
class Branching_node(Node):
    """Branching nodes have children, either a list of nodes
    or a list of two-item tuples, one per witness, for slicing
    into the original token array
    """
    children: list
    processed: bool
    absolute_offsets: list

@dataclass
class Leaf_node(Node):
    """Leaf nodes may be aligned or non-aligned and have string values"""
    string: str
    aligned: bool

def traverse_tree(r):
    """Depth-first traversal, return list of all nodes
    TODO: Replace list with generator?

    Push root (input) onto stack (just once)
    Loop: Pop top of stack, process, push all children onto stack
        NB: If children are A, B, C, A should be leftmost after being added
    Exit when stack is empty
    """
    _l = [] # list of nodes to return
    _s = deque() # extendleft() and popleft() to manipulate
    _s.appendleft(r) # root is special case
    while _s:
        current_node = _s.popleft()
        _l.append(current_node)
        if isinstance(current_node, Branching_node):
            # sequence of children should be added with leftmost at left edge
            # extendleft() reverses, so we reverse ourselves to retain original order
            _s.extendleft(current_node.children[::-1])
    return _l

# create tree root and make nodes accessible by id property
node_by_id = {}
root = Branching_node(id=0, processed=True, children=[], absolute_offsets=[])
node_by_id[0] = root
id = 0

# # First tier
# # Add branching node before first aligned node, if needed
# block0_start_positions = largest_blocks[finished[0].path[-1]][1]
# if block0_start_positions != first_absolute_token_by_witness:
#     id += 1
#     leading_unaligned_tokens = [(i, j) for i, j in zip(first_absolute_token_by_witness, block0_start_positions)]
#     root.children.append(Branching_node(id=id, processed=False, children=leading_unaligned_tokens))
#
# # Add all aligned (leaf) nodes to root node,
# # each preceded by an unaligned branching node
# for index, end_token_offset in enumerate(finished[0].path[::-1]):
#     # Add branching node for unaligned tokens
#     if index > 0:
#         id += 1
#         unaligned_pointers = []
#         current_block = largest_blocks[end_token_offset]
#         preceding_block = largest_blocks[finished[0].path[::-1][index - 1]]
#         for i in range(len(witnesses)):
#             unaligned_start_token_offset = preceding_block[1][i] + preceding_block[0]
#             unaligned_end_token_offset = current_block[1][i]
#             unaligned_pointers.append((unaligned_start_token_offset, unaligned_end_token_offset))
#         root.children.append(Branching_node(id=id, processed=False, children=unaligned_pointers))
#     # Add leaf node for aligned tokens
#     id += 1
#     block_length = largest_blocks[end_token_offset][0]
#     start_token_offset = end_token_offset - block_length
#     tokens = token_array[start_token_offset: end_token_offset]
#     root.children.append(Leaf_node(id=id, aligned=True, string=" ".join(tokens)))

def expand_branching_node(_parent, _largest_blocks, _finished, _first_absolute_token_by_witness, _witnesses, _original_offsets):
    print(f"{_finished[0]=}")
    # id is global unique node identifier (FIXME: replace with closure [or monad?])
    # _parent is node to expand, i.e., parent of nodes we create here
    # 1. Add branching node before first aligned node, if needed (optional only for tier 0)
    # 2. Add all non-branching and branching nodes
    # 3. Add branching node after last aligned node, if needed (optional only for tier 0)
    # Void; modifies tree in place
    global id
    _block0_start_positions = _largest_blocks[_finished[0].path[-1]][1]
    _block0_end_positions = [_largest_blocks[_finished[0].path[-1]][1][i] + _largest_blocks[_finished[0].path[-1]][0] for i in range(len(_witnesses))]
    print(f"{_largest_blocks[_finished[0].path[-1]]=}")
    print(f"{_block0_start_positions=}")
    print(f"{_block0_end_positions=}")

    # Add branching node before first aligned node, if needed (optional only for tier 0)
    if _block0_start_positions != _first_absolute_token_by_witness:
        id += 1
        _leading_unaligned_tokens = [(i, j) for i, j in zip(_first_absolute_token_by_witness, _block0_start_positions)]
        _new_node = Branching_node(id=id, processed=False, children=_leading_unaligned_tokens, absolute_offsets=_original_offsets[0:len(_witnesses)])
        _parent.children.append(_new_node)
        node_by_id[id] = _new_node

    # Add all aligned (leaf) nodes to root node,
    # each preceded by an unaligned branching node
    for _index, _end_token_offset in enumerate(_finished[0].path[::-1]):
        # Add branching node for unaligned tokens
        if _index > 0:
            id += 1
            _unaligned_pointers = []
            _current_block = _largest_blocks[_end_token_offset]
            _preceding_block = _largest_blocks[_finished[0].path[::-1][_index - 1]]
            for i in range(len(_witnesses)):
                # FIXME: Replace value of children with pointers into global (not local) token array
                _unaligned_start_token_offset = _preceding_block[1][i] + _preceding_block[0]
                _unaligned_end_token_offset = _current_block[1][i]
                _unaligned_pointers.append((_unaligned_start_token_offset, _unaligned_end_token_offset))
            _new_node = Branching_node(id=id, processed=False, children=_unaligned_pointers, absolute_offsets=_original_offsets[0:len(_witnesses)])
            node_by_id[id] = _new_node
            _parent.children.append(_new_node)
        # Add leaf node for aligned tokens
        id += 1
        _block_length = _largest_blocks[_end_token_offset][0]
        _start_token_offset = _end_token_offset - _block_length
        _tokens = token_array[_start_token_offset: _end_token_offset]
        _new_node = Leaf_node(id=id, aligned=True, string=" ".join(_tokens))
        node_by_id[id] = _new_node
        _parent.children.append(_new_node)

    # Add branching node after last aligned node:
    #   START HERE 2022-11-01:
    #       To process trailing tokens we need to find start and
    #           end offsets for each witness
    #       We currently find end offset for branching node we're expanding
    #       To find start offset for trailing tokens we try (!) to get
    #           end offset of preceding branching node (there must be one)
    #           plus length of intervening leaf node plus 1
    #           The start offset must be less than the end node
    #   TODO: Optional for tier 0 because it can end in an alignment
    #   Optional for other tiers because if there is no alignment,
    #       we’ve already added this as the unaligned beginning
    #       BUT we should never see these because we only invoke
    #       the expand_branching_node() function if there are blocks
    # Start position of trailing unaligned tokens is start position of
    #   last aligned ones (which we get from:
    #       end position of last unaligned + 1
    #           + length of last aligned
    _last_leaf_length = len(_parent.children[-1].string.split())
    _trailing_start_positions = [_parent.children[-2].children[i][1] + 1 + _last_leaf_length for i in range(len(_witnesses))]
    print(f"{_trailing_start_positions=}")
    # id += 1
    # _trailing_token_offsets = [(_trailing_start_positions[i], _block0_end_positions[i]) for i in range(len(_witnesses))]
    # _new_node = Branching_node(id=id, processed=False, children=_trailing_token_offsets, absolute_offsets=_original_offsets[0:len(_witnesses)])
    # _parent.children.append(_new_node)

expand_branching_node(root, largest_blocks, finished, first_absolute_token_by_witness, witnesses, root.children)
# 2022-10-18 Resume here
# Traverse tree tier by tier
# Expand branching nodes where the value of "processed" is False
#   and toggle value of "processed" to True
# TODO: Some current parameters (e.g., count of witnesses) should be global constants.

_finished[0]=BeamOption(score=80421, path=(12790, 12783, 12776, 12770, 12756, 12728, 12722, 12718, 12708, 12693, 12677, 12649, 12637, 12617, 12604, 12589, 12580, 12558, 12536, 12525, 12501, 12496, 12484, 12458, 12442, 12411, 12396, 12374, 12310, 12292, 12286, 12272, 12215, 12197, 12121, 12099, 12092, 12073, 12008, 11970, 11964, 11957, 11942, 11927, 11908, 11899, 11892, 11862, 11853, 11847, 11837, 11831, 11799, 11787, 11769, 11681, 11663, 11619, 11568, 11560, 11539, 11508, 11486, 11468, 11466, 11457, 11449, 11418, 11384, 11381, 11372, 11368, 11338, 11320, 11294, 11289, 11278, 11257, 11244, 11218, 11114, 11109, 11074, 10989, 10915, 10881, 10847, 10831, 10817, 10809, 10776, 10700, 10556, 10530, 10484, 10460, 10412, 10367, 10024, 9949, 9945, 9932, 9886, 9867, 9863, 9824, 9816, 9803, 9787, 9715, 9613, 9604, 9592, 9585, 9570, 9552, 9519, 9504, 9431, 9419, 9387, 9307, 9302, 9296, 9131, 9129, 9091, 9006, 8979, 8971, 8782, 8778, 8766, 8682, 8641, 8637, 8374, 8300, 8231, 8171, 8135, 8119, 8079, 

In [949]:
result = traverse_tree(root)
for node in result:
    # only branching nodes have a processed property
    if isinstance(node, Branching_node) and not node.processed:
        # Micro-witnesses might have no full-depth blocks. If so, expand into
        # leaf nodes and don't initiate beam search
        # FIXME: This could happen, at least in theory, on tier 0 if there are
        #   no full-depth blocks. Here we check only on lower tiers, but we need
        #   to fix it for tier 0, too.
        # ###
        # START HERE 2022-10-29
        # ###
        # FIXME: tier_witnesses is the last time we see offsets into the global token array,
        #   but we need them for unaligned portions of lower tiers. Pass the global offsets
        #   into the function that does the expansion. Simplify (or better document) the
        #   function first?
        tier_witnesses = [token_array[i:j] for i, j in node.children]
        tier_token_array, tier_token_membership_array, tier_token_witness_offset_array = create_token_array(tier_witnesses)
        # print(f"{tier_token_witness_offset_array=}")
        tier_suffix_array = SuffixArray(tier_token_array, unit=UNIT_BYTE)
        tier_lcp_array = tier_suffix_array._LCP_values
        # print(tier_suffix_array)
        tier_blocks = create_blocks(tier_suffix_array, tier_token_membership_array, tier_witnesses, tier_lcp_array)
        tier_frequent_sequences = create_blocks(tier_suffix_array,tier_token_membership_array, tier_witnesses, tier_lcp_array)
        tier_largest_blocks = find_longest_sequences(tier_frequent_sequences, tier_suffix_array)
        # print(len(tier_witnesses)) # should always be 6
        # print(tier_token_membership_array)
        # print(tier_largest_blocks)
        if tier_largest_blocks:
            tier_block_offsets_by_witness, tier_witness_offsets_to_blocks, tier_first_token_offset_in_block_by_witness, tier_first_absolute_token_by_witness, tier_score_by_block \
                = prepare_for_beam_search(tier_witnesses, tier_token_membership_array, tier_largest_blocks)
            tier_initial = [BeamOption(score=0, path=())] # one-item list
            tier_options, _ = perform_beam_search_step(tier_witnesses, tier_largest_blocks, tier_block_offsets_by_witness, tier_witness_offsets_to_blocks, tier_score_by_block)
            tier_finished = [] # options that cannot go further
            tier_counter = 0
            while tier_options: # no more options means that we're done
                # print(f"{tier_options=}")
                # TODO: The beam size at the moment is a magic number; can we rationalize it?
                tier_options, tier_end_of_life \
                    = perform_beam_search_step(tier_witnesses, tier_largest_blocks, tier_block_offsets_by_witness, tier_witness_offsets_to_blocks, tier_score_by_block, _beam_options=tier_options, _beta=20)
                tier_finished.extend(tier_end_of_life) # add any options that cannot go further
                print(tier_counter, len(tier_options), len(tier_finished))
                tier_counter += 1
            tier_finished = list(set(tier_finished)) # TODO: Remove this because we'll sort later?
            # print(tier_finished[0])
            # Set processed property and expand node
            expand_branching_node(node, tier_largest_blocks, tier_finished, tier_first_absolute_token_by_witness, tier_witnesses, node.children)
        node.processed = True

0 0 1
_finished[0]=BeamOption(score=24, path=(2,))
_largest_blocks[_finished[0].path[-1]]=(2, [0, 5, 10, 19, 28, 36])
_block0_start_positions=[0, 5, 10, 19, 28, 36]
_block0_end_positions=[2, 7, 12, 21, 30, 38]
_trailing_start_positions=[3, 8, 13, 22, 31, 39]
0 0 2
_finished[0]=BeamOption(score=10, path=(2,))
_largest_blocks[_finished[0].path[-1]]=(1, [1, 7, 12, 17, 25, 27])
_block0_start_positions=[1, 7, 12, 17, 25, 27]
_block0_end_positions=[2, 8, 13, 18, 26, 28]
_trailing_start_positions=[3, 9, 14, 19, 27, 29]
0 0 1
_finished[0]=BeamOption(score=18, path=(3,))
_largest_blocks[_finished[0].path[-1]]=(2, [1, 6, 11, 16, 21, 25])
_block0_start_positions=[1, 6, 11, 16, 21, 25]
_block0_end_positions=[3, 8, 13, 18, 23, 27]
_trailing_start_positions=[4, 9, 14, 19, 24, 28]
0 0 1
_finished[0]=BeamOption(score=8, path=(2,))
_largest_blocks[_finished[0].path[-1]]=(1, [1, 6, 11, 16, 21, 27])
_block0_start_positions=[1, 6, 11, 16, 21, 27]
_block0_end_positions=[2, 7, 12, 17, 22, 28]
_trailing_star

In [950]:
# Visualize the tree
# Create digraph and add root node
tree = graphviz.Digraph(format="svg")
tree.node(str(root.id), label="Root")
# Expand tree recursively
def populate_tree(_digraph, _parent): # void
    for n in _parent.children:
        # print(n.id)
        if isinstance(n, Leaf_node):
            label = n.string
        else:
            label = repr(n.children)
        _digraph.node(str(n.id), label=label)
        _digraph.edge(str(_parent.id), str(n.id))
populate_tree(tree, root)
for child in root.children:
    if isinstance(child, Branching_node) and child.processed == False:
        populate_tree(tree, child)
svg_tree = tree.render()
display(SVG(svg_tree))

In [951]:
for w, r in enumerate(node_by_id[991].absolute_offsets):
    print(w, token_array[r[0]:r[1]])

0 ['enter', 'on', 'the', 'copious']
1 ['enter', 'on', 'the', 'copious']
2 ['enter', 'on', 'the', 'copious']
3 ['enter', 'on', 'the', 'copious']
4 ['give', 'the']
5 ['give', 'the']


In [952]:
print(node_by_id[991])

Branching_node(id=991, children=[(0, 2), (5, 7), (10, 12), (15, 17), (20, 21), (23, 24)], processed=False, absolute_offsets=[(554, 558), (13341, 13345), (26192, 26196), (39255, 39259), (53086, 53088), (67317, 67319)])
